Importing libraries

In [6]:
import chess
import chess.pgn
import sys
import datetime
import time

Config data

In [7]:
piece_values = {
    chess.PAWN: 100, 
    chess.KNIGHT: 280, 
    chess.BISHOP: 320, 
    chess.ROOK: 479, 
    chess.QUEEN: 929, 
    chess.KING: 60000
    }

pst = {
    chess.PAWN: (   0,   0,   0,   0,   0,   0,   0,   0,
            78,  83,  86,  73, 102,  82,  85,  90,
             7,  29,  21,  44,  40,  31,  44,   7,
           -17,  16,  -2,  15,  14,   0,  15, -13,
           -26,   3,  10,   9,   6,   1,   0, -23,
           -22,   9,   5, -11, -10,  -2,   3, -19,
           -31,   8,  -7, -37, -36, -14,   3, -31,
             0,   0,   0,   0,   0,   0,   0,   0),
    chess.KNIGHT: ( -66, -53, -75, -75, -10, -55, -58, -70,
            -3,  -6, 100, -36,   4,  62,  -4, -14,
            10,  67,   1,  74,  73,  27,  62,  -2,
            24,  24,  45,  37,  33,  41,  25,  17,
            -1,   5,  31,  21,  22,  35,   2,   0,
           -18,  10,  13,  22,  18,  15,  11, -14,
           -23, -15,   2,   0,   2,   0, -23, -20,
           -74, -23, -26, -24, -19, -35, -22, -69),
    chess.BISHOP: ( -59, -78, -82, -76, -23,-107, -37, -50,
           -11,  20,  35, -42, -39,  31,   2, -22,
            -9,  39, -32,  41,  52, -10,  28, -14,
            25,  17,  20,  34,  26,  25,  15,  10,
            13,  10,  17,  23,  17,  16,   0,   7,
            14,  25,  24,  15,   8,  25,  20,  15,
            19,  20,  11,   6,   7,   6,  20,  16,
            -7,   2, -15, -12, -14, -15, -10, -10),
    chess.ROOK: (  35,  29,  33,   4,  37,  33,  56,  50,
            55,  29,  56,  67,  55,  62,  34,  60,
            19,  35,  28,  33,  45,  27,  25,  15,
             0,   5,  16,  13,  18,  -4,  -9,  -6,
           -28, -35, -16, -21, -13, -29, -46, -30,
           -42, -28, -42, -25, -25, -35, -26, -46,
           -53, -38, -31, -26, -29, -43, -44, -53,
           -30, -24, -18,   5,  -2, -18, -31, -32),
    chess.QUEEN: (   6,   1,  -8,-104,  69,  24,  88,  26,
            14,  32,  60, -10,  20,  76,  57,  24,
            -2,  43,  32,  60,  72,  63,  43,   2,
             1, -16,  22,  17,  25,  20, -13,  -6,
           -14, -15,  -2,  -5,  -1, -10, -20, -22,
           -30,  -6, -13, -11, -16, -11, -16, -27,
           -36, -18,   0, -19, -15, -15, -21, -38,
           -39, -30, -31, -13, -31, -36, -34, -42),
    chess.KING: (   4,  54,  47, -99, -99,  60,  83, -62,
           -32,  10,  55,  56,  56,  55,  10,   3,
           -62,  12, -57,  44, -67,  28,  37, -31,
           -55,  50,  11,  -4, -19,  13,   0, -49,
           -55, -43, -52, -28, -51, -47,  -8, -50,
           -47, -42, -43, -79, -64, -32, -29, -32,
            -4,   3, -14, -50, -57, -18,  13,   4,
            17,  30,  -3, -14,   6,  -1,  40,  18),
}


King Safety Helper

In [16]:
def king_attackers(board, king_square, enemy_color):
    attackers = len(board.attackers(enemy_color, king_square))
    return attackers

Endgame Helper

In [42]:
def is_endgame(board):
        total_material = 0
        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece:
                total_material += piece_values[piece.piece_type]
        if total_material <= 2400 and not board.has_castling_rights(chess.WHITE) and not board.has_castling_rights(chess.BLACK):
            return True
        else:
            return False

Evaluate the position

In [39]:
def evaluate_board(board):
    evaluation = 0
    total_material = 0
    king_safety_weight = 1.34
    king_safety_white = 0
    king_safety_black = 0

    if board.is_repetition(2):
        if board.turn == chess.WHITE:
            evaluation -= 8
        else:
            evaluation += 8

    #Simple material evaluation
    for piece_type, value in piece_values.items(): #Check for each square on the board
        for square in board.pieces(piece_type, chess.WHITE) | board.pieces(piece_type, chess.BLACK):
            piece = board.piece_at(square) #Get the piece at that square
            if piece: 
                value = piece_values[piece.piece_type] #Get the value of that piece
                if piece.piece_type != chess.KING: #Avoid king in endgame eval
                    total_material += value
                else:
                    #King safety evaluation
                    if piece.color == chess.WHITE:
                        king_safety_white = king_attackers(board, square, chess.BLACK)
                    else:
                        king_safety_black = king_attackers(board, square, chess.WHITE)

                if piece.color == chess.WHITE: 
                    evaluation += value 
                else: 
                    evaluation -= value 
                #PST bonus/malus
                _pst = {}
                if piece.color == chess.WHITE:
                    _pst = pst[piece.piece_type]
                    evaluation += _pst[square]
                else:
                    _pst = pst[piece.piece_type]
                    evaluation -= _pst[chess.square_mirror(square)]
                if piece.piece_type in (chess.KNIGHT, chess.BISHOP):
                    if piece.color == chess.WHITE and square in [chess.B1, chess.G1, chess.C1, chess.F1]:
                        evaluation -= 15

    #king safety evaluation
    if is_endgame(board):
        king_safety_weight = 0.46

    evaluation -= int(king_safety_black * king_safety_weight)
    evaluation += int(king_safety_white * king_safety_weight)

                
    return evaluation
        
    

Move ordering

In [35]:
def move_score(board, move):
    score = 0
    if board.is_capture(move):
        attacker = board.piece_at(move.from_square)
        victim = board.piece_at(move.to_square)
        # Handle en passant where victim is not on destination square
        if victim is None and board.is_en_passant(move):
            victim = chess.Piece(chess.PAWN, not board.turn)
        if victim:
            score += (piece_values[victim.piece_type] * 10) - piece_values[attacker.piece_type]
    if board.gives_check(move):
        score += 50
    if move.promotion:
        score += 200
    return score

Transposition Table

In [18]:
TT = {}

Negamax

In [ ]:
def negamax(board, depth, alpha, beta): #simple negamax implementation
    MATE_SCORE = 100000
    key = (board._transposition_key(), depth)
    if key in TT:
        return TT[key]
    if board.is_checkmate():
        return -MATE_SCORE + depth
    
    #If draw by stalemate, insufficient material, or fifty-move rule
    if board.is_stalemate() or board.is_insufficient_material() or board.can_claim_fifty_moves() or board.is_seventyfive_moves() or board.can_claim_threefold_repetition() or board.is_fivefold_repetition():
        return 0

    if depth == 0 or board.is_game_over():
        return evaluate_board(board)
    
    moves = list(board.legal_moves)
    moves.sort(key=lambda m: move_score(board, m), reverse=True)

    for move in moves:
        board.push(move)
        score = -negamax(board, depth-1, -beta, -alpha)
        board.pop()
        if score > alpha:
            alpha = score
        
        if alpha >= beta:
            break

    TT[key] = score

        
    return alpha

Find best move

In [4]:
def find_best_move(board, depth):
    best_move = None
    alpha = float('-inf')
    beta = float('inf')

    for move in board.legal_moves:
        board.push(move)
        score = -negamax(board, depth-1, -beta, -alpha)
        board.pop()

        if score > alpha:
            alpha = score
            best_move = move
    return best_move

Game loop

In [45]:
#Main game loop
prompt = input("Choose the game type: Bot vs Human (1), Bot vs Bot (2)")
if prompt == "1":
    #Bot vs Human
    prompt = input("Choose your color: White (w) or Black (b): ")
    #...
if prompt == "2":
    #Bot vs Bot
    game = chess.pgn.Game()
    board = chess.Board()

    game.headers["Event"] = "Test game"
    game.headers["White"] = "BotBreeze White"
    game.headers["Black"] = "BotBreeze Black"
    game.headers["Result"] = "*"
    game.headers["Date"] = datetime.datetime.now().strftime("%Y.%m.%d")
    game.headers["Site"] = "chess.driftbreeze.com"
    
    prompt = input("Choose bot difficulty (search depth, e.g., 2, 3, 4): ")
    depth = int(prompt)
    print(f"Starting Bot vs Bot game with depth {depth}...\n")
    number = 1
    node = game
    while not board.is_game_over():
        time_start = time.perf_counter()
        if board.is_game_over():
            break
        move = find_best_move(board, depth)
        board.push(move)
        time_end = time.perf_counter()
        print(f"Time taken: {time_end - time_start:.2f} seconds")
        print(f"{board} \n White plays {number}... {move}\n")
        node = node.add_variation(move)

        time_start = time.perf_counter()
        if board .is_game_over():
            break
        move = find_best_move(board, depth)
        board.push(move)
        time_end = time.perf_counter()
        print(f"Time taken: {time_end - time_start:.2f} seconds")
        print(f"{board} \n Black plays {number}... {move}\n")
        node = node.add_variation(move)
        number += 1
    print("Game over:", board.result())
    game.headers["Result"] = board.result()
    with open("bot_vs_bot_game.pgn", "w") as pgn_file:
        pgn_file.write(str(game))

Starting Bot vs Bot game with depth 4...

Time taken: 0.00 seconds
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . N . .
P P P P P P P P
R N B Q K B . R 
 White plays 1... g1f3

Time taken: 0.00 seconds
r n b q k b n r
. p p p p p p p
. . . . . . . .
p . . . . . . .
. . . . . . . .
. . . . . N . .
P P P P P P P P
R N B Q K B . R 
 Black plays 1... a7a5

Time taken: 0.00 seconds
r n b q k b n r
. p p p p p p p
. . . . . . . .
p . . . . . . .
. . . N . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B . R 
 White plays 2... f3d4

Time taken: 13.63 seconds
r n b q k b n r
. p p p . p p p
. . . . p . . .
p . . . . . . .
. . . N . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B . R 
 Black plays 2... e7e6

Time taken: 6.46 seconds
r n b q k b n r
. p p p . p p p
. . . . p . . .
p . . . . . . .
. . P N . . . .
. . . . . . . .
P P . P P P P P
R N B Q K B . R 
 White plays 3... c2c4

Time taken: 22.19 seconds
r n b q k b n r
. p . p . p p p
. . . . p